In [1]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Orbiter:
    name: str = 'COM'
    # Seems to be a Jupyter issue where I can't have these
    # parameters use Orbiter unless the block has already run
    # correctly once. Issue only crops up on kernel restart.
    # Changed to object to fix, doesn't matter for this.
    parent: object = None
    children: List[object] = field(default_factory=list)
    num_orbits: int = field(init=False)
        
    def __repr__(self):
        if self.parent:
            return f'{self.name}({self.parent.name})'
        else:
            return f'{self.name}()'
        
    def __post_init__(self):
        self.update_orbits()
        if self.parent:
            self.parent.add_child(self)
            
    def add_parent(self, parent):
        self.parent = parent
        self.parent.add_child(self)
        self.update_orbits()
    
    def add_child(self, child):
        self.children.append(child)
    
    def update_orbits(self):
        """Update the number of orbits then propagate down to children."""
        if self.parent:
            self.num_orbits = self.parent.num_orbits + 1
            for child in self.children:
                child.update_orbits()
        else:
            self.num_orbits = 0
    
    @property
    def parent_chain(self):
        """Return list of all parents, nearest to furthest."""
        cur_parent = self.parent
        parents = []
        while cur_parent:
            parents.append(cur_parent)
            cur_parent = cur_parent.parent
        return parents

    def min_transfers(self, target):
        """Return min # of transfers to move orbit to target's parent."""
        my_parents = self.parent_chain
        target_parents = target.parent_chain
        # Find the first shared parent in my chain. This will be
        # the minimum path.
        for i, parent in enumerate(my_parents):
            if parent in target_parents:
                # Once common parent is found, add # of hops down
                # my list to the parent + number of hops back up
                # target list (i.e. index of parent)
                return i + target_parents.index(parent)
            
        # As long as COM was done right, there should always be some
        # common parent
        raise IndexError(f'No common parent between {self} and {target}')

In [2]:
def parse_input(str_in):
    # 'A)B\n B)C\n C)D\n' -> [['A', 'B'], ['B', 'C'], ['C', 'D'], ['']]
    nodes = [line.split(')') for line in str_in.split('\n')]

    # Deal with the blank last item due to the newline at file end
    if nodes[-1] == ['']:
        nodes.pop()
    
    # For each node, add the parent to the orbiters dict if it doesn't already exist,
    # then add the child. 
    orbiters = {}
    for parent_str, child_str in nodes:
        if parent_str not in orbiters:
            parent = Orbiter(name=parent_str)
            orbiters[parent_str] = parent
        else:
            parent = orbiters[parent_str]
            
        if child_str not in orbiters:
            child = Orbiter(name=child_str, parent=parent)
            orbiters[child_str] = child
        else:
            orbiters[child_str].add_parent(parent)
    return orbiters

In [3]:
# Part 1 example regression test.
# Should have 42 orbits
example_str = '''B)G
C)D
K)L
COM)B
D)E
E)F
B)C
G)H
B)G
D)I
J)K
E)J
G)H
'''
example_orbs = parse_input(example_str)
print(example_orbs)
print([f'{orb.name}:{orb.num_orbits}' for orb in example_orbs.values()])
print(sum(orb.num_orbits for orb in example_orbs.values()))

{'B': B(COM), 'G': G(B), 'C': C(B), 'D': D(C), 'K': K(J), 'L': L(K), 'COM': COM(), 'E': E(D), 'F': F(E), 'H': H(G), 'I': I(D), 'J': J(E)}
['B:1', 'G:2', 'C:2', 'D:3', 'K:6', 'L:7', 'COM:0', 'E:4', 'F:5', 'H:3', 'I:4', 'J:5']
42


In [4]:
# Part 1 solution regression test. Solution is 110190
with open('day6_input.txt', 'r') as infile:
    in_str = infile.read()

orbs = parse_input(in_str)
print(sum(orb.num_orbits for orb in orbs.values()))

110190


In [5]:
# Part 2 example. Correct # of transfers is 4.
example_str = '''COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN'''
orbs = parse_input(example_str)
print(orbs['YOU'].parent_chain)
print(orbs['SAN'].parent_chain)
print(orbs['YOU'].min_transfers(orbs['SAN']))

[K(J), J(E), E(D), D(C), C(B), B(COM), COM()]
[I(D), D(C), C(B), B(COM), COM()]
4


In [6]:
# Part 2 problem
with open('day6_input.txt', 'r') as infile:
    in_str = infile.read()

orbs = parse_input(in_str)
print(orbs['YOU'].min_transfers(orbs['SAN']))

343
